## 1. Convert mp3 to wav

In [ ]:
#moved
import librosa
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa import display
import numpy as np
import os
import seaborn as sns
from pathlib import Path

from pydub import AudioSegment

## Putting features in learning form

In [ ]:
Required_cols.pkl

In [ ]:
Bands = ['B1','B2']
Genres = ['1','2','3']
if tempo == 'L':
    tempo = 60
elif tempo == 'M':
    tempo = 90
else:
    tempo = 120

In [ ]:
col_list = []

cols = ['Genre','beat_features','tonal_features','snippet_beat_features', 'snippet_tonal_features','song_beat_features, song_tonal_features']
for col in cols:
    for i in range(1, data[col][0].shape[0]+1):
        col_list.append(str(col)+str(i))

    df = data[col].apply(pd.Series)
    df.columns = col_list

    #append cqt feature columns to data

    data = pd.concat([data,df], axis=1)
data.drop(columns = ['Genre','tonal_features','snippet_beat_features', 'snippet_tonal_features','song_beat_features, song_tonal_features'], inplace = True)

In [ ]:
feature_list = data

#### Compress CQT to show median frequencies in selected snippet

In [ ]:
Initial_beat_features = []
beat_features = []
tonal_features = []
Instrument features = []
time_features = []


In [ ]:
data['Initial_beat_features','tonal_features','beat_features','seg_mean'].shift(-1, axis = 0)

In [ ]:
feature_list['OA_spec'] = feature_list.drop(columns = ['tempo','spec_cent','spec_bw','rolloff','zcr']).median(axis = 1, skipna = True)
feature_list = feature_list[['genre','tempo','spec_cent','spec_bw','rolloff','zcr','OA_spec']]

#### Classification using Keras sequential network

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from keras import models
from keras import layers

In [ ]:
ftrs = feature_list.drop(columns = ['genre'])
Y = feature_list[['genre']]

scaler = StandardScaler()
X = scaler.fit_transform(np.array(ftrs, dtype = float))

encoder = LabelEncoder()
y = encoder.fit_transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(264, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(8, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
feature_training = model.fit(X_train,
                    y_train,
                    epochs=30,
                    batch_size=128)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

In [ ]:
predictions = model.predict(X_test)

#### Understanding variable importances learned by model

In [ ]:
import shap
explainer = shap.DeepExplainer(model, X_train[:1000])
shap_values = explainer.shap_values(X_test[:1000])

shap.summary_plot(shap_values, X_test)

In [ ]:
filename = 'genre_detector.sav'
pickle.dump(model, open(filename, 'wb'))